# Desafío

Pongamos nuevamente en práctica todo lo que aprendimos durante la clase. He puesto los 2 nuevos conjuntos de datos disponibles para descargar a continuación:

- Proyecto Desafío 1: Ventas Online - dados_vendas_clientes.json;
- Proyecto Desafío 2: Administración de Condominios - dados_locacao_imoveis.json.
Recuerda: Estos dos proyectos de tratamiento se construyeron durante el curso. Por lo tanto, considere desarrollos previos para realizar este paso final.

## Etapa 4

### Proyecto Desafío 1: Ventas Online
En los pasos anteriores ya hemos trabajado con varios tipos de datos, ahora podemos trabajar con datos de tiempo.

En la columna `Fecha de venta` tenemos fechas en el formato 'día/mes/año' (`dd/mm/AAAA`). Transforme estos datos al tipo datetime y busque una forma de visualización de subconjunto que pueda contribuir al objetivo del contexto en el que se insertan los datos.

Si no recuerdas el problema del Proyecto Desafío 1, te dejo el texto de la situación a continuación para que sea más fácil encontrar la información:

El objetivo de este proyecto es analizar los resultados de un evento con los clientes de una empresa de venta online. Se recopiló un conjunto de datos que contiene los clientes que gastaron más en productos dentro de los 5 días posteriores a la venta, que es la duración del evento. Este análisis identificará al cliente con la mayor compra esa semana, quien recibirá un premio de la tienda, y posteriormente, puede ayudar a la empresa a crear nuevas estrategias para atraer más clientes.
Copia el código

### Proyecto Desafío 2: Administración de Condominios
Al igual que en el Proyecto Desafío 1, trabajamos con todas las columnas excepto las que involucran fechas.

En las columnas `datas_de_pagamento` y `datas_combinadas_pagamento` tenemos fechas en el formato 'día/mes/año' (`dd/mm/AAAA`). Transforme estos datos al tipo datetime y busque una forma de visualización de subconjunto que pueda contribuir al objetivo del contexto en el que se insertan los datos.

# Datos anteriores

In [40]:
import pandas as pd
import numpy as np

datos = pd.read_json("data/datos_hosting.json")
datos = pd.json_normalize(datos["info_inmuebles"])

columnas = list(datos.columns)

datos = datos.explode(columnas[3:])
datos.reset_index(inplace=True, drop=True)

datos["max_hospedes"] = datos["max_hospedes"].astype(np.int64)
datos[["cantidad_baños", "cantidad_cuartos", "cantidad_camas"]] = datos[
    ["cantidad_baños", "cantidad_cuartos", "cantidad_camas"]
].astype(np.int64)
datos["evaluacion_general"] = datos["evaluacion_general"].astype(np.float64)
datos.precio = datos.precio.apply(lambda x: x.replace("$", "").replace(",", "").strip())
datos["precio"] = datos["precio"].astype(np.float64)
datos[["cuota_deposito", "cuota_limpieza"]] = datos[
    ["cuota_deposito", "cuota_limpieza"]
].map(lambda x: x.replace("$", "").replace(",", "").strip())
datos[["cuota_deposito", "cuota_limpieza"]] = datos[
    ["cuota_deposito", "cuota_limpieza"]
].astype(np.float64)
datos["descripcion_local"] = datos["descripcion_local"].str.lower()
datos["descripcion_vecindad"] = datos["descripcion_vecindad"].str.lower()
datos["descripcion_local"] = datos["descripcion_local"].str.replace(
    r"[^a-zA-Z0-9\-\']", " ", regex=True
)
datos["descripcion_local"] = datos["descripcion_local"].str.replace(
    r"(?<!\w)-(?!\w)", " ", regex=True
)
datos["descripcion_local"] = datos["descripcion_local"].str.split()
datos["comodidades"] = datos["comodidades"].str.replace(r'\{|}|"', "", regex=True)
datos["comodidades"] = datos["comodidades"].str.split(",")

# eliminar caracteres extraños
datos["descripcion_vecindad"] = datos["descripcion_vecindad"].str.replace(
    r"[^a-zA-Z0-9\-\']", " ", regex=True
)

# cuidar uso de guiones
datos["descripcion_vecindad"] = datos["descripcion_vecindad"].str.replace(
    r"(?<!\w)-(?!\w)", " ", regex=True
)

# transformar en lista
datos["descripcion_vecindad"] = datos["descripcion_vecindad"].str.split()

datos_ventas = pd.read_json("data/datos_ventas_clientes.json")
datos_ventas = pd.json_normalize(datos_ventas["dados_vendas"])

datos_inmueble = pd.read_json("data/datos_locacion_inmuebles.json")
datos_inmueble = pd.json_normalize(datos_inmueble["dados_locacao"])
columnas = list(datos_inmueble.columns)
datos_inmueble = datos_inmueble.explode(columnas[1:])
datos_inmueble.reset_index(inplace=True, drop=True)
datos_inmueble["valor_aluguel"] = datos_inmueble["valor_aluguel"].apply(
    lambda x: x.replace("$", "").replace("reais", "").replace(",", ".").strip()
)
datos_inmueble["valor_aluguel"] = datos_inmueble["valor_aluguel"].astype(np.float64)

columnas = list(datos_ventas.columns)
datos_ventas = datos_ventas.explode(columnas[1:])
datos_ventas.reset_index(inplace=True, drop=True)
datos_ventas["Valor da compra"] = datos_ventas["Valor da compra"].apply(
    lambda x: x.replace("R$", "").replace(",", ".").strip()
)
datos_ventas["Valor da compra"] = datos_ventas["Valor da compra"].astype(np.float64)

datos_ventas["Cliente"] = datos_ventas["Cliente"].str.lower()
datos_ventas["Cliente"] = datos_ventas["Cliente"].str.replace(
    r"[^a-zA-Z\s]", "", regex=True
)
datos_inmueble["apartamento"] = datos_inmueble["apartamento"].str.replace(
    "(blocoAP)", ""
)


# Desafío 1

In [41]:
datos_ventas["Data de venda"] = pd.to_datetime(
    datos_ventas["Data de venda"], format="%d/%m/%Y"
)

In [42]:
datos_ventas.head()

,Data de venda,Cliente,Valor da compra
0,2022-06-06,ana lucia,836.50
1,2022-06-06,diego armandiu,573.33
2,2022-06-06,diego armandiu,392.80
3,2022-06-06,diego armandiu,512.34
4,2022-06-07,isabely joanes,825.31


In [43]:
datos_ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Data de venda    20 non-null     datetime64[ns]
 1   Cliente          20 non-null     object        
 2   Valor da compra  20 non-null     float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 612.0+ bytes


In [44]:
total_compras = datos_ventas.groupby(["Cliente"])['Valor da compra'].sum()
total_compras

Cliente
ana lucia           836.50
diego armandiu     1478.47
isabely joanes     2329.30
joo gabriel         386.34
julya meireles     1643.74
maria julia        2086.65
paulo castro        899.16
pedro pasco         311.15
thiago fritzz       885.24
Name: Valor da compra, dtype: float64

In [45]:
datos_ventas["Data de venda"].dt.strftime('%Y-%m')

0     2022-06
1     2022-06
2     2022-06
3     2022-06
4     2022-06
5     2022-06
6     2022-06
7     2022-06
8     2022-06
9     2022-06
10    2022-06
11    2022-06
12    2022-06
13    2022-06
14    2022-06
15    2022-06
16    2022-06
17    2022-06
18    2022-06
19    2022-06
Name: Data de venda, dtype: object

In [46]:
ventas_fecha = datos_ventas.groupby(datos_ventas["Data de venda"].dt.strftime("%Y-%m"))[
    "Valor da compra"
].sum()
ventas_fecha

Data de venda
2022-06    10856.55
Name: Valor da compra, dtype: float64

# Desafío 2

In [48]:
datos_inmueble.head()

,apartamento,datas_combinadas_pagamento,datas_de_pagamento,valor_aluguel
0,A101,01/06/2022,05/06/2022,1000.0
1,A101,01/07/2022,03/07/2022,2500.0
2,A102,02/06/2022,02/06/2022,1100.0
3,A102,02/07/2022,06/07/2022,2600.0
4,B201,03/06/2022,07/06/2022,1200.0


In [ ]:
# transformar columnas en datetime
cols = datos_inmueble.columns[1:3]
datos_inmueble[cols] = datos_inmueble[cols].apply(pd.to_datetime, format='%d/%m/%Y')

- apply -> para una fila o columna
- map -> funciona en una Series

In [68]:
datos_inmueble.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   apartamento                 30 non-null     object        
 1   datas_combinadas_pagamento  30 non-null     datetime64[ns]
 2   datas_de_pagamento          30 non-null     datetime64[ns]
 3   valor_aluguel               30 non-null     float64       
dtypes: datetime64[ns](2), float64(1), object(1)
memory usage: 1.1+ KB


In [75]:
# calcular retraso pagos
datos_inmueble["retraso_pagos"] = datos_inmueble["datas_de_pagamento"] - datos_inmueble["datas_combinadas_pagamento"]

In [77]:
datos_inmueble.head()

,apartamento,datas_combinadas_pagamento,datas_de_pagamento,valor_aluguel,retraso_pagos
0,A101,2022-06-01,2022-06-05,1000.0,4 days
1,A101,2022-07-01,2022-07-03,2500.0,2 days
2,A102,2022-06-02,2022-06-02,1100.0,0 days
3,A102,2022-07-02,2022-07-06,2600.0,4 days
4,B201,2022-06-03,2022-06-07,1200.0,4 days


In [80]:
media_retrasos= datos_inmueble.groupby(["apartamento"])["retraso_pagos"].mean()

In [82]:
media_retrasos.sort_values()

apartamento
E501    0 days 12:00:00
D401    1 days 00:00:00
F602    1 days 12:00:00
A102    2 days 00:00:00
B201    2 days 00:00:00
B202    2 days 00:00:00
G702    2 days 00:00:00
H801    2 days 00:00:00
A101    3 days 00:00:00
C302    4 days 00:00:00
D402    4 days 00:00:00
E502    4 days 00:00:00
F601    4 days 00:00:00
C301    4 days 12:00:00
G701    6 days 12:00:00
Name: retraso_pagos, dtype: timedelta64[ns]

In [83]:
datos_inmueble.describe()

,datas_combinadas_pagamento,datas_de_pagamento,valor_aluguel,retraso_pagos
count,30,30,30.000000,30
mean,2022-06-23 00:00:00,2022-06-25 20:48:00,1786.666667,2 days 20:48:00
min,2022-06-01 00:00:00,2022-06-02 00:00:00,1000.000000,0 days 00:00:00
25%,2022-06-08 06:00:00,2022-06-10 00:00:00,1300.000000,1 days 00:00:00
50%,2022-06-23 00:00:00,2022-06-26 00:00:00,1650.000000,2 days 12:00:00
75%,2022-07-07 18:00:00,2022-07-11 06:00:00,2200.000000,4 days 00:00:00
max,2022-07-15 00:00:00,2022-07-20 00:00:00,2900.000000,7 days 00:00:00
std,NaN,NaN,579.377391,2 days 03:52:17.252992466
